In [1]:
# !pip install keras-xlnet

In [2]:
import os
# from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir('/kaggle/input/he_challenge_data/data'))
os.chdir('/kaggle/input/he_challenge_data/data')

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'train', 'test', 'train.csv', 'test.csv']


In [4]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,image_id,category
0,0,77
1,1,81
2,2,52
3,3,72
4,4,58


In [5]:
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet201
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [6]:
train_path='train/'
test_path='test/'

train_images=[]
train_labels=[]

for i in range(len(train_df.image_id)):
    img=cv2.imread(train_path+str(train_df.image_id[i])+'.jpg')
    img=cv2.resize(img,(224,224))
    train_images.append(img)
    train_labels.append(train_df.category[i])

In [7]:
test_images = []
for i in range(len(test_df.image_id)):
    img=cv2.imread(test_path+str(test_df.image_id[i])+'.jpg')
    img=cv2.resize(img,(224,224))
    test_images.append(img)

In [8]:
train_images=np.array(train_images)
test_images=np.array(test_images)

print('Train shape: {}'.format(train_images.shape))
print('Test shape: {}'.format(test_images.shape))

Train shape: (18540, 224, 224, 3)
Test shape: (2009, 224, 224, 3)


In [9]:
train_images=train_images.reshape(train_images.shape[0],224,224,3)
test_images=test_images.reshape(test_images.shape[0],224,224,3)

print('Train shape: {}'.format(train_images.shape))
print('Test shape: {}'.format(test_images.shape))

Train shape: (18540, 224, 224, 3)
Test shape: (2009, 224, 224, 3)


In [10]:
train_gen = ImageDataGenerator(featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(train_images,to_categorical(train_labels),random_state=42, test_size=0.1)

print('X-Train shape: {}'.format(X_train.shape))
print('X-Val shape: {}'.format(X_val.shape))
print('Y-Train shape: {}'.format(y_train.shape))
print('Y-Val shape: {}'.format(y_val.shape))

X-Train shape: (16686, 224, 224, 3)
X-Val shape: (1854, 224, 224, 3)
Y-Train shape: (16686, 103)
Y-Val shape: (1854, 103)


In [12]:
vgg=VGG16(weights='imagenet',include_top=True)
vgg.summary()

553467904/553467096 [==============================] - 35s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_p

In [13]:
model=Sequential()
for layer in vgg.layers[:-1]:
    model.add(layer)

for layer in model.layers[:]:
    layer.trainable=False
model.add(Dense(103,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       295168    
__________

In [14]:
history=model.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_val, y_val),callbacks=[EarlyStopping(min_delta=0.01)])

Train on 16686 samples, validate on 1854 samples
Epoch 1/100
16686/16686 [==============================] - 51s 3ms/step - loss: 1.2841 - acc: 0.7409 - val_loss: 0.7516 - val_acc: 0.8420
Epoch 2/100
16686/16686 [==============================] - 47s 3ms/step - loss: 0.2851 - acc: 0.9369 - val_loss: 0.3099 - val_acc: 0.9024
Epoch 3/100
16686/16686 [==============================] - 47s 3ms/step - loss: 0.0808 - acc: 0.9760 - val_loss: 0.3473 - val_acc: 0.8991


In [15]:
# from keras.applications.nasnet import NASNetLarge

# nas = NASNetLarge(input_shape = (331, 331, 3), weights='imagenet',include_top=True)
# nas.summary()

# model_nas = Sequential()
# for layer in nas.layers[:-1]:
#     model_nas.add(layer)

# for layer in model_nas.layers[:]:
#     layer.trainable=False
# model_nas.add(Dense(103,activation='softmax'))
# model_nas.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# model_nas.summary()

# history=model_nas.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_val,y_val),callbacks=[EarlyStopping(min_delta=0.01)])

In [16]:
# from keras.applications.nasnet import NASNetMobile

# nas_mob = NASNetLarge(input_shape = (331, 331, 3), weights='imagenet',include_top=True)
# nas_mob.summary()

# model_nas_mob = Sequential()
# for layer in nas_mob.layers[:-1]:
#     model_nas_mob.add(layer)

# for layer in model_nas_mob.layers[:]:
#     layer.trainable=False
# model_nas_mob.add(Dense(103,activation='softmax'))
# model_nas_mob.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# model_nas_mob.summary()

# history=model_nas_mob.fit(X_train,y_train,epochs=100,batch_size=32,validation_data=(X_val,y_val),callbacks=[EarlyStopping(min_delta=0.01)])

In [17]:
submission=pd.read_csv('sample_submission.csv')
submission.head()

,image_id,category
0,18540,0
1,18541,0
2,18542,0
3,18543,0
4,18544,0


In [18]:
test_id=[]
test_pred=[]

for i in submission.image_id:
    img=cv2.resize(cv2.imread('test/'+str(i)+'.jpg'),(224,224))
    img=np.expand_dims(img,axis=0)
    test_id.append(i)
    test_pred.append(int(model.predict_classes(img)))

In [19]:
final_submission=pd.DataFrame({'image_id':test_id,'category':test_pred})
final_submission.head()

,image_id,category
0,18540,36
1,18541,40
2,18542,97
3,18543,72
4,18544,51


In [20]:
final_submission.to_csv('/kaggle/working/final_submission.csv',index=False)